In [ ]:
!pip install evaluate simpletransformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 103.4 MB/s eta 0:00

In [ ]:
from evaluate import load
import json
import math
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import os
import re
from itertools import chain

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Model Fine Tuning

/content/drive/MyDrive/Model Fine Tuning


In [ ]:
squad_v2_metric = load("squad_v2")
ablation = "POLICYQA_BASELINE_ORIGINAL"
# ablation = "TECHQA"

In [ ]:
model_args = {
    "learning_rate": 3e-5,
    "num_train_epochs":5,
    "output_dir": f'outputs/{ablation}',
    "best_model_dir": f'outputs/{ablation}/best_model',
    "max_seq_length": 384,
    "doc_stride": 128,
    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "train_batch_size": 16,
    "save_steps": 1000,
    "do_eval": True,
    "do_train": True,
    "do_lower_steps": True,
    "verbose": False,
    "evaluate_during_training_steps": 10069,
    "evaluate_during_training_silent": True,
    "evaluate_during_training": True,
}


model_name = f"outputs/{ablation}/best_model"
model_name = f"outputs/{ablation}/best_model"
model_type = "roberta"

model = QuestionAnsweringModel(
    model_type, model_name , args=model_args, use_cuda = True
)

In [ ]:
def read_json_as_list(dir, filename: str):
  with open(f"{dir}/{filename}.json", 'r') as file:
      data = file.read()
      parsed_data = json.loads(data)
      return list(parsed_data.values())

def flatten_list_of_lists(lst):
  flattened_list = list(chain.from_iterable(lst))
  return flattened_list

def create_split(data,train_size):

  first = int(math.floor(train_size * len(data)))
  second = 16000 + int(math.floor(0.1 * len(data)))
  # third = first  + second + int(math.floor(0.1 * len(data)))

  return data[:first],data[16000:second],data[second:]

In [ ]:
with open("../Low Resource/COVID-QA.json", "r") as file:
  full_data = json.load(file)["data"]

In [ ]:
split = 4 * len(full_data) // 5
train_data, test_data = full_data[:split], full_data[split:]

In [ ]:
with open("../Low Resource/techqa-val.json", "r") as file:
  eval_data = json.load(file)["data"]

In [ ]:
for i,data in enumerate(eval_data):
    for j,para in enumerate(data["paragraphs"]):
      for k,qa in enumerate(para["qas"]):
        eval_data[i]["paragraphs"][j]["qas"][k]["answers"][0]["answer_start"] = int(eval_data[i]["paragraphs"][j]["qas"][k]["answers"][0]["answer_start"])


In [ ]:
with open("../Low Resource/test.json", "r") as file:
  test_data = json.load(file)["data"]

In [ ]:
for i,data in enumerate(test_data):
    for j,para in enumerate(data["paragraphs"]):
      for k,qa in enumerate(para["qas"]):
        test_data[i]["paragraphs"][j]["qas"][k]["answers"][0]["answer_start"] = int(test_data[i]["paragraphs"][j]["qas"][k]["answers"][0]["answer_start"])


In [ ]:
def preprocess_squad(data):
  dataset = []
  for topic in data:
      for paragraphs in topic["paragraphs"]:
        dataset.append({"context":paragraphs["context"],"qas":paragraphs["qas"]})

  return dataset

def squad_references(data):
  references = []
  for topic in data:
    for paragraphs in topic["paragraphs"]:
      for qa in paragraphs["qas"]:
        addition = {"answers":{"text":[],"answer_start":[]},"id":qa["id"]}
        for answer in qa["answers"]:
          addition["answers"]["text"].append(answer["text"])
          addition["answers"]["answer_start"].append(answer["answer_start"])

        references.append(addition)
  return references

def generated_references(data):
  references = []
  for topic in data:
    # for paragraphs in topic["paragraphs"]:
    for qa in topic["qas"]:
      addition = {"answers":{"text":[],"answer_start":[]},"id":qa["id"]}
      for answer in qa["answers"]:
        addition["answers"]["text"].append(answer["text"])
        addition["answers"]["answer_start"].append(answer["answer_start"])

      references.append(addition)
  return references

def get_preds(texts):
  predictions = []
  for key in texts:
    for id in texts[key]:
      if key == "correct_text":
        no_ans = 1.0 if texts[key][id] == "" else 0.0
        pred = texts[key][id]
      else:
        no_ans = 1.0 if texts[key][id]["truth"] == "" else 0.0
        pred = texts[key][id]["predicted"]
      predictions.append({'prediction_text':pred , 'id': id, 'no_answer_probability': no_ans})
  return predictions

In [ ]:
references = squad_references(test_data)
test_data = preprocess_squad(test_data)

In [ ]:
references = squad_references(eval_data)
eval_data = preprocess_squad(eval_data)

In [ ]:
_,texts = model.eval_model(test_data)

Running Evaluation:   0%|          | 0/521 [00:00<?, ?it/s]

In [ ]:
predictions = get_preds(texts)

In [ ]:
results = squad_v2_metric.compute(predictions=predictions, references=references)
results

{'exact': 30.081888246628132,
 'f1': 57.653206905796665,
 'total': 4152,
 'HasAns_exact': 30.081888246628132,
 'HasAns_f1': 57.653206905796665,
 'HasAns_total': 4152,
 'best_exact': 30.081888246628132,
 'best_exact_thresh': 0.0,
 'best_f1': 57.653206905796644,
 'best_f1_thresh': 0.0}